# Imputacion de Datos Faltantes para la Precipitacion del Meta mediante modelacion KNN

Este Jupyter Notebook se enfoca en el desarrolo metodologico del trabajo de grado de los estudiantes Ronaldo Hernandez Mejia y Hector Freddy Velasco, en el cual se enfoca en la imputaciopn de datos faltnates para la variable de precipitacion en el departamento del Meta, Colombia. Con una gran volumen de datos de la informacion diaria de la precipitacion desde 1980 hasta el 2016, seleccionando un total de 56 Estaciones meteorologias del IDEAM.

La región del Meta es muy importante en este país por la diversidad del clima y su efecto sobre la agricultura y otros sectores económicos. La variabilidad de las precipitaciones juega un papel importante en el análisis del clima y la toma de decisiones relacionadas con  la gestión del agua y el clima. 

El objetivo principal es desarrolar un modelo basado en el algotimo K-Nearest Neighbors(KNN) para imputar los datos faltantes de precipitacion, utilizando un conjunto de datos que contiene mas de 700.000 observaciones. 

Dada la gran cantidad de datos se ha implementado una version que utiliza los cores del procesador para ejecutar el modelo en paralelo optimizando asi el tiempo de ejecucion, mejorando la eficiencia computacional y acelerar el proceso, tambien por este motivo el desarrolo en python tiene tiempos de respuesta cortos en el manejo de datos.



### instalacion de librerias

In [2]:
import subprocess
import sys
import importlib

def install_and_import(package):
    try:
        importlib.import_module(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    finally:
        globals()[package] = importlib.import_module(package)


In [4]:
packages = ['pandas', 'IPython', 'pandasgui', 'numpy', 'matplotlib','joblib','sklearn']

for package in packages:
    install_and_import(package)


ModuleNotFoundError: No module named 'pandasgui'

En caso de que no funcione instalacion manual:

In [11]:
pip install -U pip 

  Using cached pip-23.1.2-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sklearn


  Using cached sklearn-0.0.post4.tar.gz (3.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [10 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\vhect\AppData\Local\Temp\pip-install-okrvuf46\sklearn_e683a97414774989aac92237b0c0c687\setup.py", line 10, in <module>
          LONG_DESCRIPTION = f.read()
                             ^^^^^^^^
        File "c:\Users\vhect\AppData\Local\Programs\Python\Python311\Lib\encodings\cp1252.py", line 23, in decode
          return codecs.charmap_decode(input,self.errors,decoding_table)[0]
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
      UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 7: character maps to <undefined>
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: met

In [2]:
pip install wheel


  Using cached wheel-0.40.0-py3-none-any.whl (64 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
#observar donde estoy en ruta 
os.getcwd()

'c:\\Users\\vhect\\Documents\\semestre 10\\TESIS\\Proyecto_mod_precipitacion\\Codigos de Python'

### Ejemplo sencillo del funcionamiento del modelo KNN

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer


In [12]:

def impute_precipitation_for_station(station_data, n_neighbors=5):
    imputer = KNNImputer(n_neighbors=n_neighbors)
    station_data_imputed = imputer.fit_transform(station_data[['precipitation']])
    station_data['precipitation_imputed'] = station_data_imputed
    return station_data

# Crear datos de ejemplo
data = pd.DataFrame({
    'station_id': [1, 1, 2, 2, 3, 3],
    'precipitation': [1, np.nan, 3, 4, np.nan, 6]
})

stations = data['station_id'].unique()
imputed_data = pd.DataFrame()

for station in stations:
    station_data = data[data['station_id'] == station]
    station_data_imputed = impute_precipitation_for_station(station_data)
    imputed_data = imputed_data.append(station_data_imputed)

print(imputed_data)



   station_id  precipitation  precipitation_imputed
0           1            1.0                    1.0
1           1            NaN                    1.0
2           2            3.0                    3.0
3           2            4.0                    4.0
4           3            NaN                    6.0
5           3            6.0                    6.0


C:\Users\vhect\AppData\Local\Temp\ipykernel_8180\2479262008.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_data['precipitation_imputed'] = station_data_imputed
C:\Users\vhect\AppData\Local\Temp\ipykernel_8180\2479262008.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  imputed_data = imputed_data.append(station_data_imputed)
C:\Users\vhect\AppData\Local\Temp\ipykernel_8180\2479262008.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

### Lectura de datos

In [9]:
import pandas as pd
data = pd.read_csv("Datos/datos_combinados_para_modelo_con_coordenadas.txt",sep=";")
data.info()

FileNotFoundError: [Errno 2] No such file or directory: 'Datos/datos_combinados_para_modelo_con_coordenadas.txt'

## Bibliografia 

Little, R. J. A., & Rubin, D. B. (2002). Statistical Analysis with Missing Data (Vol. 333). John Wiley & Sons.